In [10]:
import torch
import os
import util
import train
import numpy as np
from tqdm import tqdm
import torch.distributions as dist

torch.cuda.set_device(1)

def loadModel(ckpt_path, model_name):
    model =train.chooseModel(model_name)

    model = model.load_from_checkpoint(ckpt_path, 
        use_pretrained=False)
    return model

model_rsample = loadModel('/u/amo-d0/grad/cgar/Projects/disaggregation/aggregation/results/rsample_final_2_pretrained_200_1e7_batchsize=8/best.ckpt', 'rsample')
model_gauss = loadModel('/u/amo-d0/grad/cgar/Projects/disaggregation/aggregation/results/gauss_final_2_pretrained_200/best.ckpt', 'gauss')
gauss_train = torch.dist(torch.tensor(310592.144538),torch.tensor(112758.4427697))
train_loader, val_loader, test_loader = util.make_loaders(batch_size = 1, mode = 'test', sample_mode='')

prob_avgs_sample = []
prob_avgs_gauss = []
prob_avgs_fit = []
xs = []
for x in range(10000,100000,10000):
    probs_sample = []
    probs_gauss = []
    probs_fit = []
    with torch.no_grad():
            for sample in tqdm(test_loader):
                image, mask, value = sample

                #estimated_values = model.pred_Out(image, mask)
                #print(value)

                #print(len(model_rsample.log_out(image,mask,value,x)))

                log = model_rsample.log_out(image,mask,value,x)[0].cpu().numpy().tolist()
                #print(len(log))
                probs_sample.extend(log)

                log = model_gauss.log_out(image,mask,value,x)[0].cpu().numpy().tolist()
                probs_gauss.extend(log)

                losses=[]
                for target in value[0]:
                        gauss = dist.Normal(torch.tensor(310592.1445), torch.tensor(112758.4427))

                        val = torch.tensor(x)

                        metric = gauss.cdf(target+ val) - gauss.cdf(target - val)
                        losses.append(metric)
                        probs_fit.extend(losses)

                #estimated_arr.extend( estimated_values[0].cpu().numpy().tolist())
                #value_arr.extend(value[0].numpy().tolist())
    prob_avgs_sample.append(np.array(probs_sample).mean())
    prob_avgs_gauss.append(np.array(probs_gauss).mean())
    prob_avgs_fit.append(np.array(probs_fit).mean())
    xs.append(x)

                

Reading GeoDataFrame...
Done
Loading all values...
Done...
1085 868.0 108.5 109


100%|██████████| 109/109 [01:58<00:00,  1.09s/it]


TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.